# PR0505. Análisis de estadísticas en dataset

In [7]:
from pyspark.sql.types import StringType, IntegerType, DoubleType, StructType, StructField 
schema = StructType([
    StructField("Index", IntegerType(), False),
    StructField("Title", StringType(), False),
    StructField("Description", StringType(), False),
    StructField("Amount(in rupees)", StringType(), False),
    StructField("Price (in rupees)", IntegerType(), True),
    StructField("location", StringType(), False), 
    StructField("Carpet Area", StringType(), False),
    StructField("Status", StringType(), False),
    StructField("Floor", StringType(), False),
    StructField("Transaction", StringType(), False),
    StructField("Furnishing", StringType(), False),
    StructField("facing", StringType(), True),
    StructField("overlooking", StringType(), True),
    StructField("Society", StringType(), True),
    StructField("Bathroom", IntegerType(), False),
    StructField("Balcony", IntegerType(), True),
    StructField("Car Parking", StringType(), True),
    StructField("Ownership", StringType(), True),
    StructField("Super Area", StringType(), True),
    StructField("Dimensions", StringType(), True),
    StructField("Plot Area", StringType(), True),
])

In [12]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("Estadisticas")
        .master("spark://spark-master:7077")
        .getOrCreate()
)

df = (
    spark
        .read
        .format("csv")
        .schema(schema)
        .option("header", "true")
        .load("/workspace/pr0505/house_prices.csv")
)

df.show(5)

+-----+--------------------+--------------------+-----------------+-----------------+--------+-----------+-------------+------------+-----------+--------------+------+--------------------+--------------------+--------+-------+-----------+--------------------+----------+----------+---------+
|Index|               Title|         Description|Amount(in rupees)|Price (in rupees)|location|Carpet Area|       Status|       Floor|Transaction|    Furnishing|facing|         overlooking|             Society|Bathroom|Balcony|Car Parking|           Ownership|Super Area|Dimensions|Plot Area|
+-----+--------------------+--------------------+-----------------+-----------------+--------+-----------+-------------+------------+-----------+--------------+------+--------------------+--------------------+--------+-------+-----------+--------------------+----------+----------+---------+
|    0|1 BHK Ready to Oc...|Bhiwandi, Thane h...|          42 Lac |             6000|   thane|   500 sqft|Ready to Move|10 o

## 1. Objetivos de ingeniería de datos (ETL)
### 1.1. Estandarización monetaria (de INR a USD)

In [13]:
from pyspark.sql.functions import col, when, split
df = (
    df.withColumn("Amount_USD",
        when(split(col("Amount(in rupees)"), " ").getItem(1) == "Lac", split(col("Amount(in rupees)"), " ").getItem(0).cast("int") * 100_000 * 0.012)
        .otherwise(split(col("Amount(in rupees)"), " ").getItem(0).cast("float") * 10_000_000 * 0.012)
     )
)
df.show(5)

+-----+--------------------+--------------------+-----------------+-----------------+--------+-----------+-------------+------------+-----------+--------------+------+--------------------+--------------------+--------+-------+-----------+--------------------+----------+----------+---------+----------+
|Index|               Title|         Description|Amount(in rupees)|Price (in rupees)|location|Carpet Area|       Status|       Floor|Transaction|    Furnishing|facing|         overlooking|             Society|Bathroom|Balcony|Car Parking|           Ownership|Super Area|Dimensions|Plot Area|Amount_USD|
+-----+--------------------+--------------------+-----------------+-----------------+--------+-----------+-------------+------------+-----------+--------------+------+--------------------+--------------------+--------+-------+-----------+--------------------+----------+----------+---------+----------+
|    0|1 BHK Ready to Oc...|Bhiwandi, Thane h...|          42 Lac |             6000|   tha

### 1.2. Estandarización de superficie

In [14]:
df = (
    df
        .withColumn("Carpet Area", split(col("Carpet Area"), " ").getField(0).cast("float") * 0.0929)
)
df.show(5)

+-----+--------------------+--------------------+-----------------+-----------------+--------+------------------+-------------+------------+-----------+--------------+------+--------------------+--------------------+--------+-------+-----------+--------------------+----------+----------+---------+----------+
|Index|               Title|         Description|Amount(in rupees)|Price (in rupees)|location|       Carpet Area|       Status|       Floor|Transaction|    Furnishing|facing|         overlooking|             Society|Bathroom|Balcony|Car Parking|           Ownership|Super Area|Dimensions|Plot Area|Amount_USD|
+-----+--------------------+--------------------+-----------------+-----------------+--------+------------------+-------------+------------+-----------+--------------+------+--------------------+--------------------+--------+-------+-----------+--------------------+----------+----------+---------+----------+
|    0|1 BHK Ready to Oc...|Bhiwandi, Thane h...|          42 Lac |   

## 2. Objetivos de análisis estadístico
### 2.1. Medidas de dispersión (varianza y desviación estándar)

In [23]:
from pyspark.sql.functions import std, variance, avg, max, median


print("Máximo: ")
df.select(max(col("Amount_USD"))).show(truncate=False)
print("Media: ")
df.select(avg(col("Amount_USD"))).show(truncate=False)
print("Mediana: ")
df.select(median(col("Amount_USD"))).show(truncate=False)
print("Varianza: ")
df.select(variance(col("Amount_USD"))).show(truncate=False)
print("Desviación estándar: ")
df.select(std(col("Amount_USD"))).show()

Máximo: 
+---------------+
|max(Amount_USD)|
+---------------+
|1.6803600384E8 |
+---------------+

Media: 
+-----------------+
|avg(Amount_USD)  |
+-----------------+
|143673.9094809754|
+-----------------+

Mediana: 
+------------------+
|median(Amount_USD)|
+------------------+
|93600.0           |
+------------------+

Varianza: 
+---------------------+
|var_samp(Amount_USD) |
+---------------------+
|2.2369053492349777E11|
+---------------------+

Desviación estándar: 
+------------------+
|   std(Amount_USD)|
+------------------+
|472959.33749477635|
+------------------+



La desviación estándar es más o menos tres veces la media. No es una locura, pero sí que es cierto que puede parecer bastante, sobre todo a la escala a la que están los datos. Confiar en el promedio en este caso no sería lo adecuado.

### 2.2. Medidas de Forma (Skewness y Kurtosis)

In [20]:
from pyspark.sql.functions import skewness, kurtosis

print("Asimetría: ")
df.select(skewness(col("Amount_USD"))).show()
print("Coeficiente de curtosis: ")
df.select(kurtosis(col("Amount_USD"))).show()

Asimetría: 
+--------------------+
|skewness(Amount_USD)|
+--------------------+
|   270.7528623538977|
+--------------------+

Coeficiente de curtosis: 
+--------------------+
|kurtosis(Amount_USD)|
+--------------------+
|   91483.85780267783|
+--------------------+



- La curva tiene un valor positivo (de 270, una barbaridad, es probable que haya algún dato mal tomado) por tanto la mayoría de las casas se concentran en los precios bajos y hay pocas casas caras, pero las que hay son muy caras, puede que en el ámbito de millones o puede que cientos de millones (la casa con mayor valor llega casi a los 200 millones de dólares).
- El valor de Curtosis es muy alto, por tanto, hay muchos valores muy concentrados, pero es extremadamente sensible a los valores extremos. Es poco probable que los datos de precio sean erróneos (ciertamente hay casas muy caros), pero sí que es cierto que estos pocos valores están distorsionando los datos. En este caso, sería una buena idea pasar los datos a escala logarítmica.

## 3. Interpretación para IA
### 3.1.- Pre-procesamiento para redes neuronales:

In [28]:
stats = df.select(
    avg("Amount_USD").alias("avg_usd"),
    std("Amount_USD").alias("std_usd")
).collect()[0]

avg_usd = stats["avg_usd"]
std_usd = stats["std_usd"]

stats = df.select(
    avg("Carpet Area").alias("avg_area"),
    std("Carpet Area").alias("std_area")
).collect()[0]

avg_area = stats["avg_area"]
std_area = stats["std_area"]

df = (
    df
        .withColumn("Amount_USD_Z_Score", (col("Amount_USD") - avg_usd) / std_usd)
        .withColumn("Carpet_Area_Z_Score", (col("Carpet Area") - avg_area) / std_area)
)
df.select("Amount_USD_Z_Score").show(5)
df.select("Carpet_Area_Z_Score").show(5)
df.select(max("Amount_USD_Z_Score")).show()

+--------------------+
|  Amount_USD_Z_Score|
+--------------------+
| -0.1972133798542577|
|-0.05512928366968411|
| 0.05143378846874608|
|-0.24034605191028896|
| 0.10217810853466522|
+--------------------+
only showing top 5 rows

+--------------------+
| Carpet_Area_Z_Score|
+--------------------+
|-0.22972093599364965|
|-0.23858078173842762|
|-0.13816919663094365|
| -0.2198766629438963|
|-0.18542170726975965|
+--------------------+
only showing top 5 rows

+-----------------------+
|max(Amount_USD_Z_Score)|
+-----------------------+
|      354.9825886086313|
+-----------------------+

